# SLEGO Project: UNSW CSE PhD Research - Alan Siu Lung Ng
https://github.com/alanntl/SLEGO-Project

In [1]:
# First, import the environment_setup module
import environment_setup

# Run the environment setup
config = environment_setup.setup_environment()

Starting environment setup...
Getting environment configuration...
Detecting environment...
Environment detected: Local Jupyter
Setting up configuration for Local Jupyter environment.
Configuration settings:
  drive_mainfolder: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project
  drive_folder: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub
  use_local_repo: True
  local_repo_path: None
  slego_env: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/slego_env_v0_0_1
  requirements_file: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/requirements.txt
  folder_path: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace
  dataspace: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/dataspace
  recordspace: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/recordspace
  functions

# SLEGO APP

In [2]:
display(config)

{'drive_mainfolder': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project',
 'drive_folder': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub',
 'use_local_repo': True,
 'local_repo_path': None,
 'slego_env': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/slego_env_v0_0_1',
 'requirements_file': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/requirements.txt',
 'folder_path': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace',
 'dataspace': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/dataspace',
 'recordspace': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/recordspace',
 'functionspace': '/Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/functionspace',
 'knowledgespace': '/Users/an/Library/CloudStorage/OneDrive-P

In [ ]:
import slegoapp
import panel as pn

# Kill all existing servers
pn.state.kill_all_servers()

# Create and run the new app
slego_app = slegoapp.SLEGOApp(config)
slego_app.run()

INFO:slegoapp:Initializing SLEGOApp...
INFO:slegoapp:Configuration paths:
INFO:slegoapp:  folder_path: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace
INFO:slegoapp:  functionspace: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/functionspace
INFO:slegoapp:  dataspace: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/dataspace
INFO:slegoapp:  recordspace: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/recordspace
INFO:slegoapp:  knowledgespace: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/knowledgespace
INFO:slegoapp:  ontologyspace: /Users/an/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/SLEGO-Project/slegospace/ontologyspace
INFO:slegoapp:Initializing widgets...
INFO:slegoapp:Python files available for selection: ['f-sentiment_analysis.py', 'func_autogluon.py

Launching server at http://localhost:58906


In [84]:
import rdflib
from rdflib import Graph, URIRef
from pyvis.network import Network
import logging

def visualize_sparql_network(sparql_query, ttl_file_path, output_html_path):
    # Set up logging
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger(__name__)

    # Function to create a shortened label
    def short_label(uri):
        return uri.split('/')[-1]

    # Load RDF graph
    g = Graph()
    try:
        g.parse(ttl_file_path, format='turtle')
    except Exception as e:
        logger.error(f"Error loading RDF file: {e}")
        exit(1)

    # Debug to ensure the RDF graph is loaded
    logger.debug(f"RDF graph loaded with {len(g)} triples.")
    for triple in g:
        logger.debug(triple)

    # Create a Pyvis network visualization
    net = Network(
        notebook=True, height="1000px", width="100%",
        bgcolor="#ffffff", font_color="black",
        directed=True, cdn_resources='remote'
    )

    # Set visualization options (same as before)
    net.set_options("""
    var options = {
        "nodes": {
            "shape": "box",
            "font": {
                "size": 14,
                "face": "Tahoma"
            }
        },
        "edges": {
            "arrows": {
                "to": {
                    "enabled": true,
                    "scaleFactor": 1
                }
            },
            "smooth": {
                "type": "continuous"
            }
        },
        "layout": {
            "improvedLayout": true,
            "hierarchical": {
                "enabled": false
            }
        },
        "physics": {
            "enabled": false
        }
    }
    """)

    # Execute the query
    try:
        results = g.query(sparql_query)
    except Exception as e:
        logger.error(f"Error executing SPARQL query: {e}")
        exit(1)

    if not results:
        logger.warning("SPARQL query returned no results.")
        exit(1)
    else:
        logger.debug(f"SPARQL query returned {len(results)} results.")
        for row in results:
            logger.debug(f"Subject: {row.get('subject', 'N/A')}, Predicate: {row.predicate}, Object: {row.object}")

    # Function to add a node with specified style
    def add_styled_node(node, is_central=False):
        label = short_label(str(node))
        style = {"shape": "box"}
        if is_central:
            style["color"] = "#808080"  # Red color for central nodes
            style["size"] = 40  # Larger size for central nodes
        else:
            style["color"] = "#808080"  # Light green color for other nodes
            style["size"] = 30
        net.add_node(str(node), label=label, title=str(node), **style)

    # Identify central nodes from the SPARQL query
    central_nodes = set()
    all_nodes = set()
    for row in results:
        subject = row.get('subject', None) or row[0]
        predicate = row.predicate
        object_ = row.object

        all_nodes.add(str(subject))
        if isinstance(object_, URIRef):
            all_nodes.add(str(object_))

    # Consider nodes that appear as both subject and object as central
    for node in all_nodes:
        if any(str(row.subject) == node and isinstance(row.object, URIRef) for row in results) and \
           any(isinstance(row.object, URIRef) and str(row.object) == node for row in results):
            central_nodes.add(node)

    # Add nodes and edges to the network
    added_nodes = set()
    for row in results:
        subject = row.get('subject', None) or row[0]
        predicate = row.predicate
        object_ = row.object

        # Add subject node if not already added
        if str(subject) not in added_nodes:
            add_styled_node(subject, is_central=(str(subject) in central_nodes))
            added_nodes.add(str(subject))

        # Add object node if it's a URI and not already added
        if isinstance(object_, URIRef) and str(object_) not in added_nodes:
            add_styled_node(object_, is_central=(str(object_) in central_nodes))
            added_nodes.add(str(object_))

        # Add edge if object is a URI
        if isinstance(object_, URIRef):
            edge_label = short_label(str(predicate))
            net.add_edge(str(subject), str(object_), label=edge_label, title=str(predicate))

    # Save the graph
    net.save_graph(output_html_path)
    logger.info(f"Graph saved as '{output_html_path}'. Open this file in a web browser to view the interactive visualization.")

# Example usage
if __name__ == "__main__":
    # SPARQL query for Liquidity
    query = """
                PREFIX dbr: <http://example.org/class/>
                PREFIX rdf1: <http://example.org/link/>

                SELECT ?subject ?predicate ?object
                WHERE {
                {
                    dbr:Liquidity ?predicate ?object .
                    BIND(dbr:Liquidity AS ?subject)
                }
                UNION
                {
                    dbr:ShareVolumeTraded ?predicate ?object .
                    BIND(dbr:ShareVolumeTraded AS ?subject)
                }
                }
    """

    visualize_sparql_network(
        sparql_query=query,
        ttl_file_path='slegospace/ontologyspace/hfd.ttl',
        output_html_path='sparql_network_graph.html'
    )


INFO:__main__:Graph saved as 'sparql_network_graph.html'. Open this file in a web browser to view the interactive visualization.


In [98]:
import rdflib
from rdflib import Graph, URIRef
from pyvis.network import Network
import logging
import webbrowser
import os

def visualize_sparql_network(sparql_query, ttl_file_path, output_html_path):
    # Set up logging
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger(__name__)

    # Function to create a shortened label
    def short_label(uri):
        return uri.split('/')[-1]

    # Load RDF graph
    g = Graph()
    try:
        g.parse(ttl_file_path, format='turtle')
    except Exception as e:
        logger.error(f"Error loading RDF file: {e}")
        exit(1)

    # Debug to ensure the RDF graph is loaded
    logger.debug(f"RDF graph loaded with {len(g)} triples.")
    for triple in g:
        logger.debug(triple)

    # Create a Pyvis network visualization
    net = Network(
        notebook=True, height="1000px", width="100%",
        bgcolor="#ffffff", font_color="black",
        directed=True, cdn_resources='remote'
    )

    # Set visualization options (same as before)
    net.set_options("""
    var options = {
        "nodes": {
            "shape": "box",
            "font": {
                "size": 14,
                "face": "Tahoma"
            }
        },
        "edges": {
            "arrows": {
                "to": {
                    "enabled": true,
                    "scaleFactor": 1
                }
            },
            "smooth": {
                "type": "continuous"
            }
        },
        "layout": {
            "hierarchical": {
                "enabled": true,
                "levelSeparation": 250,
                "nodeSpacing": 200,
                "treeSpacing": 300,
                "blockShifting": true,
                "edgeMinimization": true,
                "parentCentralization": true,
                "direction": "LR",
                "sortMethod": "hubsize"
            }
        },
        "physics": {
            "enabled": false
        }
    }
    """)

    # Execute the query
    try:
        results = g.query(sparql_query)
    except Exception as e:
        logger.error(f"Error executing SPARQL query: {e}")
        exit(1)

    if not results:
        logger.warning("SPARQL query returned no results.")
        exit(1)
    else:
        logger.debug(f"SPARQL query returned {len(results)} results.")
        for row in results:
            logger.debug(f"Subject: {row.get('subject', 'N/A')}, Predicate: {row.predicate}, Object: {row.object}")

    # Function to add a node with specified style
    def add_styled_node(node, is_central=False):
        label = short_label(str(node))
        style = {"shape": "box"}
        if is_central:
            style["color"] = "#808080"  # Gray color for central nodes
            style["size"] = 40  # Larger size for central nodes
        else:
            style["color"] = "#808080"  # Gray color for other nodes
            style["size"] = 30
        net.add_node(str(node), label=label, title=str(node), **style)

    # Identify central nodes from the SPARQL query
    central_nodes = set()
    all_nodes = set()
    for row in results:
        subject = row.get('subject', None) or row[0]
        predicate = row.predicate
        object_ = row.object

        all_nodes.add(str(subject))
        if isinstance(object_, URIRef):
            all_nodes.add(str(object_))

    # Consider nodes that appear as both subject and object as central
    for node in all_nodes:
        if any(str(row.subject) == node and isinstance(row.object, URIRef) for row in results) and \
           any(isinstance(row.object, URIRef) and str(row.object) == node for row in results):
            central_nodes.add(node)

    # Add nodes and edges to the network
    added_nodes = set()
    for row in results:
        subject = row.get('subject', None) or row[0]
        predicate = row.predicate
        object_ = row.object

        # Add subject node if not already added
        if str(subject) not in added_nodes:
            add_styled_node(subject, is_central=(str(subject) in central_nodes))
            added_nodes.add(str(subject))

        # Add object node if it's a URI and not already added
        if isinstance(object_, URIRef) and str(object_) not in added_nodes:
            add_styled_node(object_, is_central=(str(object_) in central_nodes))
            added_nodes.add(str(object_))

        # Add edge if object is a URI
        if isinstance(object_, URIRef):
            edge_label = short_label(str(predicate))
            net.add_edge(str(subject), str(object_), label=edge_label, title=str(predicate))

    # Save the graph
    net.save_graph(output_html_path)
    logger.info(f"Graph saved as '{output_html_path}'.")
    
    # Open the saved HTML file in the default web browser
    try:
        webbrowser.open('file://' + os.path.realpath(output_html_path))
        logger.info("Graph opened in your default web browser.")
    except Exception as e:
        logger.error(f"Error opening the graph in web browser: {e}")
        logger.info("Please open the HTML file manually to view the visualization.")

# Example usage
if __name__ == "__main__":
    # SPARQL query for Liquidity and ShareVolumeTraded
    query = """
                PREFIX dbr: <http://example.org/class/>
                PREFIX rdf1: <http://example.org/link/>

                SELECT ?subject ?predicate ?object
                WHERE {
                {
                    dbr:IntradayMetric  ?predicate ?object .
                    BIND(dbr:IntradayMetric  AS ?subject)
                }
                UNION
                {
                    dbr:ShareVolumeTraded ?predicate ?object .
                    BIND(dbr:ShareVolumeTraded AS ?subject)
                }
                }
    """

    visualize_sparql_network(
        sparql_query=query,
        ttl_file_path='slegospace/ontologyspace/hfd.ttl',
        output_html_path='sparql_network_graph.html'
    )

INFO:__main__:Graph saved as 'sparql_network_graph.html'.
INFO:__main__:Graph opened in your default web browser.


In [97]:
# SPARQL query for the entire graph
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?subject ?predicate ?object ?subject_type ?object_type
WHERE {
    ?subject ?predicate ?object .
    OPTIONAL { ?subject rdf:type ?subject_type }
    OPTIONAL { ?object rdf:type ?object_type }
}
"""

visualize_sparql_network(
    sparql_query=query,
    ttl_file_path='slegospace/ontologyspace/hfd.ttl',
    output_html_path='full_graph_visualization.html'
)

INFO:__main__:Graph saved as 'full_graph_visualization.html'.
INFO:__main__:Graph opened in your default web browser.
